In [17]:
import pandas as pd
from orbit.utils.dataset import load_iclaims

# Load sample data
df = load_iclaims()
df["week"] = pd.to_datetime(df["week"])
df = df.rename(columns={"week": "date", "claims": "value"})

print(df.head())

        date      value  trend.unemploy  trend.filling  trend.job     sp500  \
0 2010-01-03  13.386595        0.219882      -0.318452   0.117500 -0.417633   
1 2010-01-10  13.624218        0.219882      -0.194838   0.168794 -0.425480   
2 2010-01-17  13.398741        0.236143      -0.292477   0.117500 -0.465229   
3 2010-01-24  13.137549        0.203353      -0.194838   0.106918 -0.481751   
4 2010-01-31  13.196760        0.134360      -0.242466   0.074483 -0.488929   

        vix  
0  0.122654  
1  0.110445  
2  0.532339  
3  0.428645  
4  0.487404  


In [25]:
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data

# Initialize the DLT model
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,  # Weekly seasonality
)

# Fit the model
model.fit(df)

2025-01-13 17:04:56 - orbit - INFO - Sampling (CmdStanPy) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 225 and samples(per chain): 25.
2025-01-13 17:04:56 - orbit - INFO - Compiling stan model:/opt/anaconda3/lib/python3.12/site-packages/orbit/stan/dlt.stan. ETA 3 - 5 mins.


ValueError: Failed to compile Stan model '/opt/anaconda3/lib/python3.12/site-packages/orbit/stan/dlt.stan'. Console:
clang++ -pipe   -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare -D_REENTRANT -ffp-contract=off  -O3 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials -DNO_FPRINTF_OUTPUT     -O3  -c -x c -include stan/lib/stan_math/lib/sundials_6.1.1/include/stan_sundials_printf_override.hpp stan/lib/stan_math/lib/sundials_6.1.1/src/nvector/serial/nvector_serial.c -o stan/lib/stan_math/lib/sundials_6.1.1/src/nvector/serial/nvector_serial.o
stan/lib/stan_math/lib/sundials_6.1.1/src/nvector/serial/nvector_serial.c:19:10: fatal error: 'stdio.h' file not found
#include <stdio.h>
         ^~~~~~~~~
1 error generated.
make: *** [stan/lib/stan_math/make/libraries:65: stan/lib/stan_math/lib/sundials_6.1.1/src/nvector/serial/nvector_serial.o] Error 1

Command ['make', 'STANCFLAGS+=--filename-in-msg=dlt.stan', '/opt/anaco

In [ ]:
import pandas as pd
from orbit.utils.dataset import load_iclaims
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data

# Load sample data
df = load_iclaims()
df["week"] = pd.to_datetime(df["week"])
df = df.rename(columns={"week": "date", "claims": "value"})

print(df.head())

# Split data into train and test
train_df = df.iloc[:-20]  # Use all but the last 20 rows for training
test_df = df.iloc[-20:]  # Use the last 20 rows for testing

# Initialize the DLT model
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,  # Weekly seasonality
)

# Fit the model
model.fit(df=train_df)

# Make predictions
forecast = model.predict(df=test_df)

# Plot actual vs predicted values
plot_predicted_data(
    training_actual_df=train_df,
    predicted_df=forecast,
    date_col="date",
    actual_col="value",
    pred_col="prediction",
    test_actual_df=test_df,
)


In [ ]:
from cmdstanpy import install_cmdstan

# Install or overwrite the existing installation
install_cmdstan(overwrite=True)


In [ ]:
# Predict future values
predictions = model.predict(df=data)

# Plot the predicted data
plot_predicted_data(data, predictions, date_col="date", actual_col="value", pred_col="prediction")

In [ ]:
# Extract prediction intervals
predictions[["prediction", "prediction_5", "prediction_95"]].head()

In [ ]:
# Initialize a model with custom parameters
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,
    damped=True,  # Damped trend
)
model.fit(train_df=data)

In [ ]:
from orbit.models import KTR

# Use Kernel Trend Regression (KTR) with custom priors
model = KTR(
    response_col="value",
    date_col="date",
    seasonality=52,
    level_knot_prior=0.5,
)
model.fit(train_df=data)

In [ ]:
from orbit.diagnostics.metrics import smape, rmse

# Calculate SMAPE and RMSE
true_values = data["value"]
predicted_values = predictions["prediction"]
print("SMAPE:", smape(true_values, predicted_values))
print("RMSE:", rmse(true_values, predicted_values))

In [ ]:
# Add external regressors (e.g., economic indicators)
data["recession"] = [1 if x % 12 < 3 else 0 for x in range(len(data))]  # Simulated example

# Initialize the model with regressors
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,
    regressor_col=["recession"]
)
model.fit(train_df=data)